# **Library Import + Set Config**

Notebook written by: Ankita Ghosh, Ivan Milev, Sayan Deb Sarkar, Stefan Stefanache 

In [6]:
import os
import os.path as osp
import pandas as pd
import numpy as np
import random
import umap
import pickle
import multiprocessing as mp
import json
from glob import glob 

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

In [3]:
# Change these paths according to your need
data_dir = '../data'
seed = 42
sample_size = 100
num_components = 3

# **Utility Functions**

In [4]:
def load_pkl_data(filename):
    with open(filename, 'rb') as handle:
        data_dict = pickle.load(handle)
    return data_dict

def write_pkl_data(data_dict, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

def sample_values_spaced(low, high, num_values):
    values = np.linspace(low, low, num=num_values, dtype=int)
    values = np.unique(values)
    
    while len(values) < 100:
        extra_values = np.random.randint(low, high + 1, size=(num_values - len(values)))
        values = np.unique(np.concatenate((values, extra_values)))
    return values

def load_json(filename):
    file = open(filename)
    data = json.load(file)
    file.close()
    return data

def write_json(data_dict, filename):
    json_obj = json.dumps(data_dict, indent=4)

    with open(filename, "w") as outfile:
        outfile.write(json_obj)

# **Load Data + Merge Annotations with Color Palette**

In [13]:
annotations_file=osp.join(data_dir, 'seeds00000-10000.pkl')
# metadata_file = osp.join(data_dir, 'color_palette00000-99999.csv')

# metadata = pd.read_csv(metadata_file)
annotations = load_pkl_data(annotations_file)

df_scores_vectors = glob(data_dir + '/scores_modifications*')
df_scores = pd.DataFrame()
for df_score in df_scores_vectors:
    df_score_l = pd.read_csv(df_score)
    df_scores = pd.concat([df_scores, df_score_l], axis=0)

df_scores = df_scores.sort_values('Final Score', ascending=False).reset_index()
df_scores = df_scores.groupby('Feature').first()
df_scores['Method General'] = df_scores['Method'].str.split('_').apply(lambda x: x[0])

separation_vectors = glob(data_dir + '/*_separation_vector*.csv')
df_sep_vecs = pd.DataFrame()
for df_sep_vec in separation_vectors:
    if 'modifications' not in df_sep_vec:
        df_sep_vec_l = pd.read_csv(df_sep_vec)
        df_sep_vecs = pd.concat([df_sep_vecs, df_sep_vec_l], axis=0)

df_sep_vecs['Method General'] = df_sep_vecs['Method'].str.split('_').apply(lambda x: x[0])

df_scores = df_scores.merge(df_sep_vecs, left_on=['Feature', 'Variable', 'Method General'], right_on=['Feature', 'Variable', 'Method General'], how='left')
df_scores = df_scores[df_scores['Separation Vector'].notnull()]
print(df_scores[['Feature', 'Variable', 'Method General', 'Final Score']].head(10), df_scores.shape)

                Feature Variable Method General  Final Score
0            ALPHA BLUE    color     ShapleyVec        0.809
1        AMERICAN ROAST    color     ShapleyVec        0.934
2             ANTIQUITY    color     StyleSpace        0.959
3            AWARD BLUE    color   InterfaceGAN        0.638
4              CHAI TEA    color     ShapleyVec        0.936
5          CRUSADE KING    color     StyleSpace        0.942
6              FALU RED    color     ShapleyVec        0.950
7  FIREBIRD TAIL LIGHTS    color     ShapleyVec        0.930
8             FUJI SNOW    color     StyleSpace        0.937
9       NECRON COMPOUND    color     ShapleyVec        0.903 (18, 40)


In [14]:
best_scores = df_scores.sort_values('Final Score', ascending=False).groupby('Feature').first()
best_scores

,index,Method_x,Variable,Broad,Optimal lambda,Total Score,Score lambda 1,Score lambda 2,Score lambda 3,Score lambda 4,...,Score lambda 23,Score lambda 24,Score lambda 25,Method General,Space,Method_y,Subfolder,Classes,Separation Vector,Performance
Feature,,,,,,,,,,,,,,,,,,,,,
ALPHA BLUE,27,ShapleyVec_True_0.25_18_color_LR,color,True,9.0,0.812,0.602,0.627,0.639,0.656,...,0.16,0.14,0.16,ShapleyVec,w,ShapleyVec_True_0.5_18_color_LR,ShapleyVec_beauty/color/,"SEED PEARL, FUJI SNOW, ORANGE ESSENTIAL, RIVIE...",[ 0.144 0. 0. 0. 0. -0.08...,NaN
AMERICAN ROAST,13,ShapleyVec_True_0.25_18_color_LR,color,True,10.0,0.917,0.865,0.859,0.885,0.897,...,0.20,0.24,0.24,ShapleyVec,w,ShapleyVec_True_0.5_18_color_LR,ShapleyVec_beauty/color/,"SEED PEARL, FUJI SNOW, ORANGE ESSENTIAL, RIVIE...",[ 0. 0. 0. 0. 0. -0.05...,NaN
ANTIQUITY,15,StyleSpace_True_10_18_color_False,color,True,15.0,0.943,0.936,0.935,0.937,0.945,...,0.08,0.08,0.08,StyleSpace,w,StyleSpace_True_40_18_color_False,StyleSpace_beauty/color/,"SEED PEARL, FUJI SNOW, ORANGE ESSENTIAL, RIVIE...",[ 0. 0. 0. 0. 0. 0. ...,NaN
AWARD BLUE,69,InterfaceGAN_LR_1_18_color_True,color,True,20.0,0.643,0.581,0.586,0.592,0.600,...,0.14,0.16,0.16,InterfaceGAN,w,InterfaceGAN_LR_0.1_18_color_True,interfaceGAN_beauty/color/,"SEED PEARL, FUJI SNOW, ORANGE ESSENTIAL, RIVIE...",[-8.47676366e-03 -1.84804634e-03 3.18864185e-...,0.26
CHAI TEA,57,ShapleyVec_True_0.5_18_color_LR,color,True,6.0,0.914,0.902,0.917,0.919,0.923,...,0.12,0.16,0.14,ShapleyVec,w,ShapleyVec_True_0.5_18_color_LR,ShapleyVec_beauty/color/,"SEED PEARL, FUJI SNOW, ORANGE ESSENTIAL, RIVIE...",[ 0. 0. 0. 0. 0. 0. ...,NaN
CRUSADE KING,103,StyleSpace_True_40_18_color_False,color,True,15.0,0.930,0.912,0.916,0.917,0.930,...,0.14,0.18,0.16,StyleSpace,w,StyleSpace_True_40_18_color_False,StyleSpace_beauty/color/,"SEED PEARL, FUJI SNOW, ORANGE ESSENTIAL, RIVIE...",[ 0. 0. 0. 0. 0. 0. ...,NaN
FALU RED,29,ShapleyVec_True_0.25_18_color_LR,color,True,14.0,0.914,0.826,0.833,0.858,0.867,...,0.82,0.82,0.84,ShapleyVec,w,ShapleyVec_True_0.5_18_color_LR,ShapleyVec_beauty/color/,"SEED PEARL, FUJI SNOW, ORANGE ESSENTIAL, RIVIE...",[ 0. 0. 0. 0. 0. -0.23...,NaN
FIREBIRD TAIL LIGHTS,55,ShapleyVec_True_0.5_18_color_LR,color,True,5.0,0.936,0.847,0.889,0.903,0.911,...,0.34,0.34,0.34,ShapleyVec,w,ShapleyVec_True_0.5_18_color_LR,ShapleyVec_beauty/color/,"SEED PEARL, FUJI SNOW, ORANGE ESSENTIAL, RIVIE...",[ 0. 0. 0. 0. 0.0674 0. ...,NaN
FUJI SNOW,75,StyleSpace_True_40_18_color_False,color,True,10.0,0.922,0.918,0.925,0.928,0.936,...,0.50,0.54,0.54,StyleSpace,w,StyleSpace_True_40_18_color_False,StyleSpace_beauty/color/,"SEED PEARL, FUJI SNOW, ORANGE ESSENTIAL, RIVIE...",[ 0. 0. 0. 0. 0. 0. ...,NaN


In [7]:
# metadata.head()

In [8]:
annotations = pd.DataFrame(annotations)
annotations['fname'] = annotations['fname'].str.split('/').apply(lambda x: x[-1])
annotations.head()

,fname,seeds,z_vectors,w_vectors,color
0,seed00000.png,0,"[[1.764052345967664, 0.4001572083672233, 0.978...","[[-0.033073805, 2.199152, 0.1471625, -0.402728...",SEED PEARL
1,seed00001.png,1,"[[1.6243453636632417, -0.6117564136500754, -0....","[[-0.26820526, 1.5636992, -0.3447469, -0.14454...",FUJI SNOW
2,seed00002.png,2,"[[-0.4167578474054706, -0.056266827226329474, ...","[[2.0551293, 0.6194114, -0.5921279, -0.0676888...",ORANGE ESSENTIAL
3,seed00003.png,3,"[[1.7886284734303186, 0.43650985051198943, 0.0...","[[0.3626459, -0.2004384, 2.5389173, -0.1788218...",RIVIERA BEACH
4,seed00004.png,4,"[[0.05056170714293955, 0.499951333237829, -0.9...","[[-0.17903857, 3.492336, 2.138563, -0.40349492...",FUJI SNOW


# **Calculate Intensity**

Average of V1 and S1 (they are the main intensity components)

In [9]:
merged_df = annotations #metadata.merge(annotations, left_on='fname', right_on='fname')
# merged_df['intensity'] = (merged_df['V1'] + merged_df['S1']) / 2.0
merged_df.head(10)

,fname,seeds,z_vectors,w_vectors,color
0,seed00000.png,0,"[[1.764052345967664, 0.4001572083672233, 0.978...","[[-0.033073805, 2.199152, 0.1471625, -0.402728...",SEED PEARL
1,seed00001.png,1,"[[1.6243453636632417, -0.6117564136500754, -0....","[[-0.26820526, 1.5636992, -0.3447469, -0.14454...",FUJI SNOW
2,seed00002.png,2,"[[-0.4167578474054706, -0.056266827226329474, ...","[[2.0551293, 0.6194114, -0.5921279, -0.0676888...",ORANGE ESSENTIAL
3,seed00003.png,3,"[[1.7886284734303186, 0.43650985051198943, 0.0...","[[0.3626459, -0.2004384, 2.5389173, -0.1788218...",RIVIERA BEACH
4,seed00004.png,4,"[[0.05056170714293955, 0.499951333237829, -0.9...","[[-0.17903857, 3.492336, 2.138563, -0.40349492...",FUJI SNOW
5,seed00005.png,5,"[[0.44122748688504143, -0.33087015189408764, 2...","[[1.8812124, 1.5100609, -0.30210596, -0.004075...",ORANGE ESSENTIAL
6,seed00006.png,6,"[[-0.3117836734875166, 0.7290039236125041, 0.2...","[[-0.28565487, 2.6148477, 0.71299654, 0.568077...",FUJI SNOW
7,seed00007.png,7,"[[1.690525703800356, -0.4659373705408328, 0.03...","[[-0.15524942, -0.13580184, -0.26550844, 0.444...",RIVIERA BEACH
8,seed00008.png,8,"[[0.09120471661981977, 1.0912827332721105, -1....","[[-0.07481922, -0.17790766, -0.33518022, -0.21...",NECRON COMPOUND
9,seed00009.png,9,"[[0.0011085547122109732, -0.2895440692501988, ...","[[0.5770087, 1.110505, -0.23896095, -0.1350926...",WOODEN CABIN


# **Different Mapping Methods to reduce 512-d to 2-d**



*   Principal Component Analysis (PCA)
*   t-distributed Stochastic Neighbor Embedding(t-SNE)
*   Uniform Manifold Approximation and Projection (UMAP)



In [37]:
# PCA
points = np.array(merged_df['w_vectors'].tolist()).squeeze()
pca = PCA(n_components = num_components, random_state = seed)
pca.fit(points)
points_transformed = pca.transform(points)
merged_df['pca_3d'] = list([list(x) for x in points_transformed.astype(float)])
merged_df.head()

,fname,seeds,z_vectors,w_vectors,color,pca_2d,pca_3d,tsne_3d,umap_3d
0,seed00000.png,0,"[[1.764052345967664, 0.4001572083672233, 0.978...","[[-0.033073805, 2.199152, 0.1471625, -0.402728...",SEED PEARL,"[0.81112224, -1.8070444]","[0.824035108089447, -1.8561749458312988, -2.03...","[4.1237173, 58.588192, 36.53986]","[6.345804, 3.3984215, 6.5413814]"
1,seed00001.png,1,"[[1.6243453636632417, -0.6117564136500754, -0....","[[-0.26820526, 1.5636992, -0.3447469, -0.14454...",FUJI SNOW,"[-0.53969693, 3.7407146]","[-0.5579930543899536, 3.8020682334899902, -2.5...","[-40.82981, 25.924463, 5.9590225]","[4.2520933, 3.9682264, 7.5821557]"
2,seed00002.png,2,"[[-0.4167578474054706, -0.056266827226329474, ...","[[2.0551293, 0.6194114, -0.5921279, -0.0676888...",ORANGE ESSENTIAL,"[0.93320227, -2.9125655]","[0.9184034466743469, -2.868119478225708, -0.44...","[-20.869707, -12.281309, -10.970484]","[5.459795, 2.6448028, 6.91313]"
3,seed00003.png,3,"[[1.7886284734303186, 0.43650985051198943, 0.0...","[[0.3626459, -0.2004384, 2.5389173, -0.1788218...",RIVIERA BEACH,"[0.5621117, 2.7881286]","[0.5575673580169678, 2.803612470626831, -1.792...","[-51.436954, -41.73785, 4.5131063]","[4.951328, 4.327596, 7.7960124]"
4,seed00004.png,4,"[[0.05056170714293955, 0.499951333237829, -0.9...","[[-0.17903857, 3.492336, 2.138563, -0.40349492...",FUJI SNOW,"[-2.8292613, 1.0246211]","[-2.8310418128967285, 1.0306297540664673, 3.08...","[58.34184, 8.984057, -0.89038736]","[4.688453, 5.2712307, 5.6012263]"


In [13]:
# # TSNE
# points = np.array(merged_df['w_vectors'].tolist()).squeeze()
# tsne = TSNE(n_jobs=mp.cpu_count(), n_components = num_components, random_state = seed)
# points_transformed = tsne.fit_transform(points)
# merged_df['tsne_3d'] = list([list(x) for x in points_transformed])
# merged_df.head()

,fname,seeds,z_vectors,w_vectors,color,pca_2d,pca_3d,tsne_3d
0,seed00000.png,0,"[[1.764052345967664, 0.4001572083672233, 0.978...","[[-0.033073805, 2.199152, 0.1471625, -0.402728...",SEED PEARL,"[0.81112224, -1.8070444]","[0.8240351, -1.856175, -2.0397165]","[4.1237173, 58.588192, 36.53986]"
1,seed00001.png,1,"[[1.6243453636632417, -0.6117564136500754, -0....","[[-0.26820526, 1.5636992, -0.3447469, -0.14454...",FUJI SNOW,"[-0.53969693, 3.7407146]","[-0.55799305, 3.8020682, -2.5367172]","[-40.82981, 25.924463, 5.9590225]"
2,seed00002.png,2,"[[-0.4167578474054706, -0.056266827226329474, ...","[[2.0551293, 0.6194114, -0.5921279, -0.0676888...",ORANGE ESSENTIAL,"[0.93320227, -2.9125655]","[0.91840345, -2.8681195, -0.4441436]","[-20.869707, -12.281309, -10.970484]"
3,seed00003.png,3,"[[1.7886284734303186, 0.43650985051198943, 0.0...","[[0.3626459, -0.2004384, 2.5389173, -0.1788218...",RIVIERA BEACH,"[0.5621117, 2.7881286]","[0.55756736, 2.8036125, -1.7924287]","[-51.436954, -41.73785, 4.5131063]"
4,seed00004.png,4,"[[0.05056170714293955, 0.499951333237829, -0.9...","[[-0.17903857, 3.492336, 2.138563, -0.40349492...",FUJI SNOW,"[-2.8292613, 1.0246211]","[-2.8310418, 1.0306298, 3.089939]","[58.34184, 8.984057, -0.89038736]"


In [38]:
# Umap
points = np.array(merged_df['w_vectors'].tolist()).squeeze()
umap_reducer = umap.UMAP(n_components = num_components, n_jobs = 1, random_state = seed)
umap_reducer.fit(points)
points_transformed = umap_reducer.transform(points)
merged_df['umap_3d'] = list([list(x) for x in points_transformed.astype(float)])
merged_df.head()

,fname,seeds,z_vectors,w_vectors,color,pca_2d,pca_3d,tsne_3d,umap_3d
0,seed00000.png,0,"[[1.764052345967664, 0.4001572083672233, 0.978...","[[-0.033073805, 2.199152, 0.1471625, -0.402728...",SEED PEARL,"[0.81112224, -1.8070444]","[0.824035108089447, -1.8561749458312988, -2.03...","[4.1237173, 58.588192, 36.53986]","[6.345804214477539, 3.3984215259552, 6.5413813..."
1,seed00001.png,1,"[[1.6243453636632417, -0.6117564136500754, -0....","[[-0.26820526, 1.5636992, -0.3447469, -0.14454...",FUJI SNOW,"[-0.53969693, 3.7407146]","[-0.5579930543899536, 3.8020682334899902, -2.5...","[-40.82981, 25.924463, 5.9590225]","[4.252093315124512, 3.968226432800293, 7.58215..."
2,seed00002.png,2,"[[-0.4167578474054706, -0.056266827226329474, ...","[[2.0551293, 0.6194114, -0.5921279, -0.0676888...",ORANGE ESSENTIAL,"[0.93320227, -2.9125655]","[0.9184034466743469, -2.868119478225708, -0.44...","[-20.869707, -12.281309, -10.970484]","[5.459794998168945, 2.6448028087615967, 6.9131..."
3,seed00003.png,3,"[[1.7886284734303186, 0.43650985051198943, 0.0...","[[0.3626459, -0.2004384, 2.5389173, -0.1788218...",RIVIERA BEACH,"[0.5621117, 2.7881286]","[0.5575673580169678, 2.803612470626831, -1.792...","[-51.436954, -41.73785, 4.5131063]","[4.951327800750732, 4.327596187591553, 7.79601..."
4,seed00004.png,4,"[[0.05056170714293955, 0.499951333237829, -0.9...","[[-0.17903857, 3.492336, 2.138563, -0.40349492...",FUJI SNOW,"[-2.8292613, 1.0246211]","[-2.8310418128967285, 1.0306297540664673, 3.08...","[58.34184, 8.984057, -0.89038736]","[4.688453197479248, 5.271230697631836, 5.60122..."


In [39]:
merged_df['umap_3d'][0]

[6.345804214477539, 3.3984215259552, 6.541381359100342]

In [44]:
merged_df['w_vectors'] = [list(x[0].astype(float)) for x in merged_df['w_vectors'].tolist()]
merged_df['w_vectors'][0]

[-0.03307380527257919,
 2.1991519927978516,
 0.14716249704360962,
 -0.402728796005249,
 -0.3192251920700073,
 1.4729201793670654,
 0.3154529631137848,
 -0.04597093164920807,
 -0.05270218476653099,
 -0.23931100964546204,
 -0.33684301376342773,
 2.4419524669647217,
 1.6391160488128662,
 -0.0012365338625386357,
 0.24100609123706818,
 0.08894571661949158,
 2.299691915512085,
 1.7963258028030396,
 -0.22383151948451996,
 -0.26804211735725403,
 -0.34680792689323425,
 0.4689982235431671,
 0.7789413332939148,
 -0.02866033837199211,
 -0.40189725160598755,
 1.2752293348312378,
 -0.1512499749660492,
 -0.4953002631664276,
 -0.14030492305755615,
 0.049373697489500046,
 -0.22881822288036346,
 1.0836354494094849,
 0.9428068399429321,
 0.07689003646373749,
 -0.06244581192731857,
 -0.09461905062198639,
 0.560248076915741,
 3.301952600479126,
 -0.27062085270881653,
 0.17830710113048553,
 0.6284207701683044,
 -0.08868083357810974,
 -0.4714738130569458,
 -0.19104169309139252,
 0.47237488627433777,
 3.21372

In [40]:
pickle.dump(pca, open(data_dir + f"pca.pkl","wb"))
pickle.dump(umap_reducer, open(data_dir + f"umap.pkl","wb"))

# Mapping File Creation
Create a Dictionary with Colors as the Keys and the vectors + mapped data sorted per color according to intensity, save as a JSON file**

In [ ]:
merged_df_sample = merged_df.head(100)
with open(data_dir + f"/3d_umap_points.json", "w") as outfile: 
    json.dump(merged_df_sample['umap_3d'].tolist(), outfile)

with open(data_dir + f"/3d_pca_points.json", "w") as outfile: 
    json.dump(merged_df_sample['pca_3d'].tolist(), outfile)

with open(data_dir + f"/512d_points.json", "w") as outfile: 
    json.dump(merged_df_sample['w_vectors'].tolist(), outfile)

In [17]:
best_scores['Separation Vector'] = [np.array([float(x.strip('[] ')) for x in sep_v.replace('\n', ' ').split(' ') if x.strip('[] ') != '']) for sep_v in best_scores['Separation Vector']]

In [19]:
umap = load_pkl_data(data_dir + '/umap.pkl')
pca = load_pkl_data(data_dir + '/pca.pkl')

directions = np.array(best_scores['Separation Vector'].tolist()).squeeze()
best_scores['umap_3d'] = list([list(x) for x in umap.transform(directions).astype(float)])
best_scores['pca_3d'] = list([list(x) for x in pca.transform(directions).astype(float)])


In [21]:
col2orig = {}
for color, vec in zip(best_scores.index, best_scores['Separation Vector']):
    col2orig[color] = list(vec.astype(float)) 
    
col2orig


{'ALPHA BLUE': [0.144,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.0812,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0241,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.1428,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0608,
  0.0,
  0.0,
  0.1586,
  0.0,
  -0.1469,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.2131,
  0.0,
  0.0,
  0.0,
  0.11,
  0.0,
  0.0674,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.1202,
  -0.1232,
  0.1581,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.0719,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.065,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.0804,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  -0.2027,
  0.0,
  0.0

In [23]:
col2umap = {k:v for k,v in zip(best_scores.index, best_scores['umap_3d'])}
col2pca = {k:v for k,v in zip(best_scores.index, best_scores['pca_3d'])}


In [24]:
with open(data_dir + f"/3d_umap_directions.json", "w") as outfile: 
    json.dump(col2umap, outfile)

with open(data_dir + f"/3d_pca_points.json", "w") as outfile: 
    json.dump(col2pca, outfile)

with open(data_dir + f"/512d_points.json", "w") as outfile: 
    json.dump(col2orig, outfile)